In [1]:
import os
import numpy as np
import pandas as pd

from pathlib import Path
from glob import glob
from dotenv import load_dotenv

In [2]:
res_dir = r'c:/Users/by003457/downloads'
rev_file = 'company_revenue_analysis_strategy01.xlsx'

In [3]:
res_dirs = [Path(x) for x in glob(os.path.join(res_dir, 'strategy01_*'))]
res_dirs

[WindowsPath('c:/Users/by003457/downloads/strategy01_aggressive'),
 WindowsPath('c:/Users/by003457/downloads/strategy01_conservative'),
 WindowsPath('c:/Users/by003457/downloads/strategy01_creative')]

In [4]:
res_datasets = []
for rd in res_dirs:
    file_path = rd / rev_file
    if not file_path.exists():
        continue
    df = pd.read_excel(file_path)
    df['result_dir'] = rd.name
    cols = df.columns.tolist()
    cols = ['result_dir'] + [col for col in cols if col != 'result_dir']
    df = df[cols]
    res_datasets.append(df)
if not res_datasets:
    raise FileNotFoundError("No result directories contained the revenue analysis file.")

In [5]:
res_datasets[0].head(2)

,result_dir,revenue,tickerSymbol,companyName,indu_desc,desc_1,desc_2,desc_3,desc_4,desc_5,p_neg20below,p_neg5to-20,p_neg5to5,p_5to20,p_above20,total_trades,p_above20_ratio
0,strategy01_aggressive,48126780,A000760,"Rifa Co.,Ltd.",Chemical Manufacturing,Manufacturing,Chemical Manufacturing,NaN,NaN,NaN,0,1,36,1,1,39,0.025641
1,strategy01_aggressive,32801730,A004090,Korea Petroleum Industries Company,"Asphalt Paving, Roofing, and Saturated Materia...",Manufacturing,Petroleum and Coal Products Manufacturing,Petroleum and Coal Products Manufacturing,"Asphalt Paving, Roofing, and Saturated Materia...",NaN,0,1,15,1,2,19,0.105263


In [6]:
combined_df = pd.concat(res_datasets, ignore_index=True)
combined_df.head(2)

,result_dir,revenue,tickerSymbol,companyName,indu_desc,desc_1,desc_2,desc_3,desc_4,desc_5,p_neg20below,p_neg5to-20,p_neg5to5,p_5to20,p_above20,total_trades,p_above20_ratio
0,strategy01_aggressive,48126780,A000760,"Rifa Co.,Ltd.",Chemical Manufacturing,Manufacturing,Chemical Manufacturing,NaN,NaN,NaN,0,1,36,1,1,39,0.025641
1,strategy01_aggressive,32801730,A004090,Korea Petroleum Industries Company,"Asphalt Paving, Roofing, and Saturated Materia...",Manufacturing,Petroleum and Coal Products Manufacturing,Petroleum and Coal Products Manufacturing,"Asphalt Paving, Roofing, and Saturated Materia...",NaN,0,1,15,1,2,19,0.105263


In [7]:
expected_profit_bins = ["p_neg20below", "p_neg5to-20", "p_neg5to5", "p_5to20", "p_above20"]
for col in expected_profit_bins:
    if col not in combined_df.columns:
        combined_df[col] = 0
if "total_trades" not in combined_df.columns:
    combined_df["total_trades"] = combined_df[expected_profit_bins].sum(axis=1)
combined_df["total_trades"] = combined_df["total_trades"].fillna(0).astype(float)
if "p_above20_ratio" not in combined_df.columns:
    combined_df["p_above20_ratio"] = np.where(
        combined_df["total_trades"] > 0,
        combined_df["p_above20"] / combined_df["total_trades"],
        np.nan,
    )
combined_df.head(2)

,result_dir,revenue,tickerSymbol,companyName,indu_desc,desc_1,desc_2,desc_3,desc_4,desc_5,p_neg20below,p_neg5to-20,p_neg5to5,p_5to20,p_above20,total_trades,p_above20_ratio
0,strategy01_aggressive,48126780,A000760,"Rifa Co.,Ltd.",Chemical Manufacturing,Manufacturing,Chemical Manufacturing,NaN,NaN,NaN,0,1,36,1,1,39.0,0.025641
1,strategy01_aggressive,32801730,A004090,Korea Petroleum Industries Company,"Asphalt Paving, Roofing, and Saturated Materia...",Manufacturing,Petroleum and Coal Products Manufacturing,Petroleum and Coal Products Manufacturing,"Asphalt Paving, Roofing, and Saturated Materia...",NaN,0,1,15,1,2,19.0,0.105263


In [8]:
profit_bin_cols = ["p_neg20below", "p_neg5to-20", "p_neg5to5", "p_5to20", "p_above20"]
ticker_column = None
if "tickerSymbol" in combined_df.columns:
    ticker_column = "tickerSymbol"
elif "ticker" in combined_df.columns:
    ticker_column = "ticker"
agg_dict = {'revenue': 'sum', 'total_trades': 'sum'}
if ticker_column:
    agg_dict[ticker_column] = 'nunique'
agg_base = combined_df.groupby('result_dir').agg(agg_dict).reset_index()
agg_base = agg_base.rename(columns={'revenue': 'total_revenue'})
if ticker_column and ticker_column in agg_base.columns:
    agg_base = agg_base.rename(columns={ticker_column: 'unique_tickers'})
else:
    agg_base['unique_tickers'] = np.nan
bin_totals = combined_df.groupby('result_dir')[profit_bin_cols].sum().reset_index()
summary_by_result = agg_base.merge(bin_totals, on='result_dir', how='left')
summary_by_result['total_trades'] = summary_by_result['total_trades'].astype(float)
summary_by_result = summary_by_result.assign(
    avg_revenue_per_trade=lambda df_: np.where(
        df_['total_trades'] > 0,
        df_['total_revenue'] / df_['total_trades'],
        np.nan,
    ),
    win_trades=lambda df_: df_[['p_5to20', 'p_above20']].sum(axis=1),
    nearflat_trades=lambda df_: df_['p_neg5to5'],
    loss_trades=lambda df_: df_[['p_neg20below', 'p_neg5to-20']].sum(axis=1),
    win_ratio=lambda df_: np.where(
        df_['total_trades'] > 0,
        df_['win_trades'] / df_['total_trades'],
        np.nan,
    ),
    high_win_ratio=lambda df_: np.where(
        df_['total_trades'] > 0,
        df_['p_above20'] / df_['total_trades'],
        np.nan,
    ),
    estimated_avg_profit_pct=lambda df_: np.where(
        df_['total_trades'] > 0,
        100
        * (
            df_['p_neg20below'] * (-0.25)
            + df_['p_neg5to-20'] * (-0.125)
            + df_['p_neg5to5'] * 0.0
            + df_['p_5to20'] * 0.125
            + df_['p_above20'] * 0.25
        )
        / df_['total_trades'],
        np.nan,
    ),
)
summary_by_result = summary_by_result.round({
    'avg_revenue_per_trade': 2,
    'win_ratio': 3,
    'high_win_ratio': 3,
    'estimated_avg_profit_pct': 2,
})
ordered_columns = [
    'result_dir',
    'unique_tickers',
    'total_trades',
    'total_revenue',
    'avg_revenue_per_trade',
    'estimated_avg_profit_pct',
    'win_trades',
    'loss_trades',
    'nearflat_trades',
    'win_ratio',
    'high_win_ratio',
    'p_neg20below',
    'p_neg5to-20',
    'p_neg5to5',
    'p_5to20',
    'p_above20',
 ]
summary_by_result[ordered_columns]

,result_dir,unique_tickers,total_trades,total_revenue,avg_revenue_per_trade,estimated_avg_profit_pct,win_trades,loss_trades,nearflat_trades,win_ratio,high_win_ratio,p_neg20below,p_neg5to-20,p_neg5to5,p_5to20,p_above20
0,strategy01_aggressive,1498,22042.0,923565338,41900.25,0.12,514,331,21197,0.023,0.002,7,324,21197,475,39
1,strategy01_conservative,197,7649.0,1305402,170.66,0.13,121,44,7484,0.016,0.002,9,35,7484,109,12
2,strategy01_creative,887,74018.0,96696067,1306.39,0.20,4939,4207,64872,0.067,0.008,120,4087,64872,4375,564


In [9]:
consistent_profit_metrics = summary_by_result[[
    'result_dir',
    'win_trades',
    'loss_trades',
    'nearflat_trades',
    'total_trades',
    'win_ratio',
    'high_win_ratio',
    'estimated_avg_profit_pct',
]].copy()
consistent_profit_metrics

,result_dir,win_trades,loss_trades,nearflat_trades,total_trades,win_ratio,high_win_ratio,estimated_avg_profit_pct
0,strategy01_aggressive,514,331,21197,22042.0,0.023,0.002,0.12
1,strategy01_conservative,121,44,7484,7649.0,0.016,0.002,0.13
2,strategy01_creative,4939,4207,64872,74018.0,0.067,0.008,0.20


In [10]:
profit_bin_distribution = summary_by_result[[
    'result_dir',
    'p_neg20below',
    'p_neg5to-20',
    'p_neg5to5',
    'p_5to20',
    'p_above20',
    'total_trades',
]].copy()
for col in ['p_neg20below', 'p_neg5to-20', 'p_neg5to5', 'p_5to20', 'p_above20']:
    profit_bin_distribution[f'pct_{col}'] = np.where(
        profit_bin_distribution['total_trades'] > 0,
        profit_bin_distribution[col] / profit_bin_distribution['total_trades'],
        np.nan,
    )
profit_bin_distribution

,result_dir,p_neg20below,p_neg5to-20,p_neg5to5,p_5to20,p_above20,total_trades,pct_p_neg20below,pct_p_neg5to-20,pct_p_neg5to5,pct_p_5to20,pct_p_above20
0,strategy01_aggressive,7,324,21197,475,39,22042.0,0.000318,0.014699,0.961664,0.021550,0.001769
1,strategy01_conservative,9,35,7484,109,12,7649.0,0.001177,0.004576,0.978429,0.014250,0.001569
2,strategy01_creative,120,4087,64872,4375,564,74018.0,0.001621,0.055216,0.876435,0.059107,0.007620


In [11]:
top_ticker_by_revenue = (
    combined_df
    .groupby(['result_dir', 'tickerSymbol', 'companyName'], dropna=False)
    .agg({
        'revenue': 'sum',
        'total_trades': 'sum',
        'p_neg20below': 'sum',
        'p_neg5to-20': 'sum',
        'p_neg5to5': 'sum',
        'p_5to20': 'sum',
        'p_above20': 'sum',
    })
    .rename(columns={'revenue': 'total_revenue'})
    .reset_index()
 )
top_ticker_by_revenue = top_ticker_by_revenue.assign(
    win_trades=lambda df_: df_[['p_5to20', 'p_above20']].sum(axis=1),
    high_win_ratio=lambda df_: np.where(
        df_['total_trades'] > 0,
        df_['p_above20'] / df_['total_trades'],
        np.nan,
    ),
    win_ratio=lambda df_: np.where(
        df_['total_trades'] > 0,
        df_['win_trades'] / df_['total_trades'],
        np.nan,
    ),
)
top_ticker_by_revenue.sort_values(['result_dir', 'total_revenue'], ascending=[True, False]).groupby('result_dir').head(5)

,result_dir,tickerSymbol,companyName,total_revenue,total_trades,p_neg20below,p_neg5to-20,p_neg5to5,p_5to20,p_above20,win_trades,high_win_ratio,win_ratio
32,strategy01_aggressive,A000760,"Rifa Co.,Ltd.",48126780,39.0,0,1,36,1,1,2,0.025641,0.051282
172,strategy01_aggressive,A004090,Korea Petroleum Industries Company,32801730,19.0,0,1,15,1,2,3,0.105263,0.157895
1415,strategy01_aggressive,A404950,DB Finance No.10 Special Purpose Acquisition C...,23753654,25.0,0,0,24,0,1,1,0.040000,0.040000
115,strategy01_aggressive,A002760,Bolak Company Limited,20818942,9.0,0,0,8,0,1,1,0.111111,0.111111
307,strategy01_aggressive,A009140,"Kyungin Electronics Co., Ltd",20723240,26.0,0,0,24,1,1,2,0.038462,0.076923
1564,strategy01_conservative,A079000,"Watos Corea Co., Ltd.",552170,11.0,0,0,0,11,0,11,0.000000,1.000000
1526,strategy01_conservative,A005500,"Samjin Pharmaceuticals Co., Ltd.",400750,9.0,0,0,2,7,0,7,0.000000,0.777778
1565,strategy01_conservative,A084670,Dongyang Express Corp.,347550,9.0,0,0,2,7,0,7,0.000000,0.777778
1571,strategy01_conservative,A094800,Mirae Asset Asia Pacific Real Estate 1 Investm...,325630,132.0,0,8,109,15,0,15,0.000000,0.113636
1584,strategy01_conservative,A155900,Badaro No. 19 Ship Investment Company,282680,150.0,0,0,150,0,0,0,0.000000,0.000000
